# Ансамблевые модели

## Задача классификации 

В этом практическом задании вы научитесь работать с ансамблевыми моделями. Мы начнем с задачи классификации итальянского вина на предмет его пренадлежности к одному из трех видов. Загрузите датасет `Wine Data Database` с помощью функции `load_wine` из модуля `sklearn.datasets`.

In [2]:
from sklearn.datasets import load_wine
from sklearn.datasets import load_boston

boston = load_boston()
X, y = load_wine(return_X_y=True)

Модель случайного леса для классификации представлена классом `RandomForestClassifier` из модуля `sklearn.ensemble`. Конструктор этого класса содержит аргумент `n_estimators`, который соответствует колличеству базовых алгоритмов в случайном лесе. Целью этого задания будет настройка этого параметра. Сравните модели случайных лесов с различным числом базовых алгоритмов `{1, 5, 10, 20}`. Что происходит с качеством случайного леса на тестовых данных при увеличении этого числа? Ответом на это задание `answer1` является лучшая оценка качества модели, округленная до трех знаков после запятой. Используйте `accuracy` как метрику качества и скользящий контроль `cross_val_score` как метод оценки качества модели. Установите параметр `cv = StratifiedKFold(4)`. Возьмите среднее значение оценки качества. Для каждой из моделей случайного леса используете `random_state=42` при создании нового экземпляра.

### *РЕШЕНИЕ*

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold

# Создаём словарь мооделей с разным количеством n_estimators
estimators = {1: RandomForestClassifier(n_estimators=1, random_state=42), 
              5: RandomForestClassifier(n_estimators=5, random_state=42), 
              10: RandomForestClassifier(n_estimators=10, random_state=42), 
              20: RandomForestClassifier(n_estimators=20, random_state=42)}

# Установливаем параметр cv = StratifiedKFold(4)
cv = StratifiedKFold(4)

# Проводим оценку качества модели
scores = {n: cross_val_score(model, X, y, cv=cv, scoring='accuracy') for n, model in estimators.items()}

# Находим среднее значение оценки качества
best_model = max(scores, key=lambda n: scores[n].mean())
answer1 = scores[best_model].mean()

answer1 = round(answer1, 3)

print(answer1)


0.972


Далее сравните модель градиентного бустинга `GradientBoostingClassifier` из `sklearn.ensemble` с логистической регрессией `LogisticRegression` из `sklearn.linear_model` на этой выборке. Используете параметр `random_state=42` при создании экземпляров классов. Какая из моделей работает лучше? Приведите лучшую оценку, округленную до трех знаков после запятой, в качестве ответа `answer2` на это задание. Какие выводы из этого можно сделать?

### *РЕШЕНИЕ*

In [4]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

models = {'GradientBoosting': GradientBoostingClassifier(random_state=42),
          'LogisticRegression': LogisticRegression(random_state=42, max_iter=10000)}

scores = {n: cross_val_score(model, X, y, cv=cv, scoring='accuracy') for n, model in models.items()}

best_model = max(scores, key=lambda n: scores[n].mean())
answer2 = scores[best_model].mean()

answer2 = round(answer2, 3)

print(answer2)

0.967


## Задача регрессии

Загрузите уже известную вам выборку `Boston House Prices` и разделите ее случайным образом на тренировочную и тестовую выборку. Для этого используете функцию `train_test_split` с параметрами `random_state=54` и `test_size=0.33`. Мы будем сравнивать 4 модели: `RandomForestRegressor`, `GradientBoostingRegressor` из `sklearn.ensemble`, а так же Гребневую регрессию и ЛАССО (`Ridge`, `Lasso` из `sklearn.linear_model`). Обучите каждую модель на тренировочной выборке с параметром `random_state=42` в конструкторе. Какая из моделей показывает наименьшее значение среднеквадратической ошибки на тестовых данных? В качестве ответа `answer3` приведите это значение, округленное до двух цифр после запятой.

### *РЕШЕНИЕ*

In [5]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(boston.data, boston.target, test_size=0.33, random_state=54)

# Обучаем RandomForestRegressor на тестовой выборке
rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train)

# Обучаем GradientBoostingRegressor на тестовой выборке
gb = GradientBoostingRegressor(random_state=42)
gb.fit(X_train, y_train)

# Обучаем Ridge на тестовой выборке
ridge = Ridge(random_state=42)
ridge.fit(X_train, y_train)

# Обучаем Lasso на тестовой выборке
lasso = Lasso(random_state=42)
lasso.fit(X_train, y_train)

# Предсказываем на тестовой выборке 
rf_pred = rf.predict(X_test)
gb_pred = gb.predict(X_test)
ridge_pred = ridge.predict(X_test)
lasso_pred = lasso.predict(X_test)

# Вычисляем MSE для каждой модели
rf_mse = mean_squared_error(y_test, rf_pred)
gb_mse = mean_squared_error(y_test, gb_pred)
ridge_mse = mean_squared_error(y_test, ridge_pred)
lasso_mse = mean_squared_error(y_test, lasso_pred)

# Находим наименьшее значение среднеквадратической ошибки
mse_values = [rf_mse, gb_mse, ridge_mse, lasso_mse]
model_names = ['Random Forest', 'Gradient Boosting', 'Ridge', 'Lasso']
min_mse = min(mse_values)
min_mse_index = mse_values.index(min_mse)
best_model = model_names[min_mse_index]

answer3 = round(min_mse, 2)

print(f"The best model is {best_model} with MSE: {answer3}")


The best model is Gradient Boosting with MSE: 8.54


# Строка с ответами

In [6]:
output = """Best score (random forest) {0:.3f}
Best score (other algorithms) {1:.3f}
Best score (regression) {2:.2f}"""
print(output.format(answer1, answer2, answer3))

Best score (random forest) 0.972
Best score (other algorithms) 0.967
Best score (regression) 8.54
